In [ ]:
!pip install --upgrade lxml

In [12]:
from yahoo_fin import stock_info as si

print(si.get_live_price("aapl"))
 
# or Amazon
print(si.get_live_price("amzn"))
 


# get quote table back as a data frame
print(si.get_quote_table("amzn", dict_result = False))
 
# or get it back as a dictionary (default)
#si.get_quote_table("aapl")

# get most active stocks on the day
print(si.get_day_most_active())
 
# get biggest gainers
print(si.get_day_gainers())
 
# get worst performers
print(si.get_day_losers())

289.07000732421875
2286.0400390625
                   attribute                        value
0              1y Target Est                      2544.15
1              52 Week Range          1,626.03 - 2,475.00
2                        Ask               2,276.95 x 800
3                Avg. Volume                    6.202e+06
4          Beta (5Y Monthly)                         1.25
5                        Bid              2,269.95 x 1400
6                Day's Range          2,258.25 - 2,362.44
7                  EPS (TTM)                        20.94
8              Earnings Date  Jul 23, 2020 - Jul 27, 2020
9           Ex-Dividend Date                          NaN
10  Forward Dividend & Yield                    N/A (N/A)
11                Market Cap                        1.14T
12                      Open                       2336.8
13            PE Ratio (TTM)                       109.19
14            Previous Close                         2474
15               Quote Price         

   Symbol                                   Name  Price (Intraday)  Change  \
0      WY                   Weyerhaeuser Company             17.97   -3.90   
1    OPCH               Option Care Health, Inc.             11.88   -2.42   
2    NCLH    Norwegian Cruise Line Holdings Ltd.             13.84   -2.56   
3    SHEN  Shenandoah Telecommunications Company             45.65   -7.86   
4     APA                     Apache Corporation             11.28   -1.80   
..    ...                                    ...               ...     ...   
95   SHLX         Shell Midstream Partners, L.P.             13.50   -1.19   
96    CFX                     Colfax Corporation             23.78   -2.01   
97   FLEX                              Flex Ltd.              9.00   -0.76   
98   NGVT                   Ingevity Corporation             47.88   -4.04   
99   DOYU   DouYu International Holdings Limited              7.00   -0.59   

    % Change        Volume  Avg Vol (3 month)    Market Cap  PE

In [10]:

from lxml import html
import requests
from time import sleep
import json
import argparse
from random import randint

def parse_finance_page(ticker):
  """
  Grab financial data from NASDAQ page
  
  Args:
    ticker (str): Stock symbol
  
  Returns:
    dict: Scraped data
  """
  key_stock_dict = {}
  headers = {
        "Accept":"text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,image/apng,*/*;q=0.8",
        "Accept-Encoding":"gzip, deflate",
        "Accept-Language":"en-GB,en;q=0.9,en-US;q=0.8,ml;q=0.7",
        "Connection":"keep-alive",
        "Host":"www.nasdaq.com",
        "Referer":"http://www.nasdaq.com",
        "Upgrade-Insecure-Requests":"1",
        "User-Agent":"Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/64.0.3282.119 Safari/537.36"
  }

  # Retrying for failed request
  for retries in range(5):
    try:    
      url = "http://www.nasdaq.com/symbol/%s"%(ticker)
      response = requests.get(url, headers = headers, verify=False)
      
      if response.status_code!=200:
        raise ValueError("Invalid Response Received From Webserver")

      print("Parsing %s"%(url))
      # Adding random delay
      sleep(randint(1,3))   
      parser = html.fromstring(response.text)
      xpath_head = "//div[@id='qwidget_pageheader']//h1//text()"
      xpath_key_stock_table = '//div[@class="row overview-results relativeP"]//div[contains(@class,"table-table")]/div'
      xpath_open_price = '//b[contains(text(),"Open Price:")]/following-sibling::span/text()'
      xpath_open_date = '//b[contains(text(),"Open Date:")]/following-sibling::span/text()'
      xpath_close_price = '//b[contains(text(),"Close Price:")]/following-sibling::span/text()'
      xpath_close_date = '//b[contains(text(),"Close Date:")]/following-sibling::span/text()'
      xpath_key = './/div[@class="table-cell"]/b/text()'
      xpath_value = './/div[@class="table-cell"]/text()'

      raw_name = parser.xpath(xpath_head)
      key_stock_table =  parser.xpath(xpath_key_stock_table)
      raw_open_price = parser.xpath(xpath_open_price)
      raw_open_date = parser.xpath(xpath_open_date)
      raw_close_price = parser.xpath(xpath_close_price)
      raw_close_date = parser.xpath(xpath_close_date)

      company_name = raw_name[0].replace("Common Stock Quote & Summary Data","").strip() if raw_name else ''
      open_price =raw_open_price[0].strip() if raw_open_price else None
      open_date = raw_open_date[0].strip() if raw_open_date else None
      close_price = raw_close_price[0].strip() if raw_close_price else None
      close_date = raw_close_date[0].strip() if raw_close_date else None

      # Grabbing ans cleaning keystock data
      for i in key_stock_table:
        key = i.xpath(xpath_key)
        value = i.xpath(xpath_value)
        key = ''.join(key).strip() 
        value = ' '.join(''.join(value).split()) 
        key_stock_dict[key] = value

      nasdaq_data = {

              "company_name":company_name,
              "ticker":ticker,
              "url":url,
              "open price":open_price,
              "open_date":open_date,
              "close_price":close_price,
              "close_date":close_date,
              "key_stock_data":key_stock_dict
      }
      return nasdaq_data

    except Exception as e:
      print("Failed to process the request, Exception:%s"%(e))

if __name__=="__main__":


  ticker = 'amzn'
  print("Fetching data for %s"%(ticker))
  scraped_data = parse_finance_page(ticker)
  print("Writing scraped data to output file")

  with open('%s-summary.json'%(ticker),'w') as fp:
    json.dump(scraped_data,fp,indent = 4,ensure_ascii=False)

Fetching data for amzn


C:\Users\ADMIN\.conda\envs\project\lib\site-packages\urllib3\connectionpool.py:1004: InsecureRequestWarning: Unverified HTTPS request is being made to host 'www.nasdaq.com'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning,
C:\Users\ADMIN\.conda\envs\project\lib\site-packages\urllib3\connectionpool.py:1004: InsecureRequestWarning: Unverified HTTPS request is being made to host 'www.nasdaq.com'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning,


Parsing http://www.nasdaq.com/symbol/amzn
Writing scraped data to output file


In [8]:
#Python program to scrape website  
#and save quotes from website 
import requests 
from bs4 import BeautifulSoup 
import csv 
import re

#Python program to scrape website  
#and save quotes from website 
import requests 
from bs4 import BeautifulSoup 
import csv 



def parsenews(vardata):  
    URL =vardata.rstrip().lstrip().strip()# "https://www.hindustantimes.com/delhi-news/protest-at-delhi-s-jama-masjid-against-citizenship-act-4-metro-stations-closed-in-area/story-q7vKj5IUdIKMExw5eGBfxI.html"
    #URL ="https://www.hindustantimes.com/delhi-news/protest-at-delhi-s-jama-masjid-against-citizenship-act-4-metro-stations-closed-in-area/story-q7vKj5IUdIKMExw5eGBfxI.html"
    print(repr(URL))
    headers = {
        "Accept":"text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,image/apng,*/*;q=0.8",
        "Accept-Encoding":"gzip, deflate",
        "Accept-Language":"en-GB,en;q=0.9,en-US;q=0.8,ml;q=0.7",
        "Connection":"keep-alive",
        "Host":"www.nasdaq.com",
        "Referer":"http://www.nasdaq.com",
        "Upgrade-Insecure-Requests":"1",
        "User-Agent":"Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/64.0.3282.119 Safari/537.36"
  }
    r = requests.get(URL, headers = headers, verify=False)
    print(r.text)
    soup = BeautifulSoup(r.content, 'html.parser') 
  
    quotes=[]  # a list to store quotes 
  
    table = soup.find('span', attrs = {'class':'symbol-page-header__pricing-price'}) 
    print(table)
    for ik in table:
        print(ik)


parsenews('http://www.nasdaq.com/symbol/amzn')


'http://www.nasdaq.com/symbol/amzn'


C:\Users\ADMIN\.conda\envs\project\lib\site-packages\urllib3\connectionpool.py:1004: InsecureRequestWarning: Unverified HTTPS request is being made to host 'www.nasdaq.com'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning,
C:\Users\ADMIN\.conda\envs\project\lib\site-packages\urllib3\connectionpool.py:1004: InsecureRequestWarning: Unverified HTTPS request is being made to host 'www.nasdaq.com'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning,


<!DOCTYPE html>
<html lang="en" dir="ltr" prefix="content: http://purl.org/rss/1.0/modules/content/  dc: http://purl.org/dc/terms/  foaf: http://xmlns.com/foaf/0.1/  og: http://ogp.me/ns#  rdfs: http://www.w3.org/2000/01/rdf-schema#  schema: http://schema.org/  sioc: http://rdfs.org/sioc/ns#  sioct: http://rdfs.org/sioc/types#  skos: http://www.w3.org/2004/02/skos/core#  xsd: http://www.w3.org/2001/XMLSchema# ">
  <head>
    <meta charset="utf-8" />
<meta itemprop="acquia_lift:content_title" content="AMZN" />
<meta itemprop="acquia_lift:content_type" content="page" />
<meta itemprop="acquia_lift:page_type" content="content page" />
<meta itemprop="acquia_lift:context_language" content="en" />
<meta itemprop="acquia_lift:content_section" content="market-activity/stocks" />
<meta itemprop="acquia_lift:content_keywords" content="" />
<meta itemprop="acquia_lift:post_id" content="" />
<meta itemprop="acquia_lift:content_uuid" content="" />
<meta itemprop="acquia_lift:published_date" conten